In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [86]:
from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import GradientBoostingRegressor
import xgboost as xgb

In [4]:
auctions = pd.read_csv("data/auctions.csv.gzip", parse_dates=['date'])
auctions.columns = ['date', 'ref_hash', 'ref_type_id', 'source_id']

In [5]:
auctions_first = auctions[auctions['date'] < '2019-04-21 00:00:00.000000']
auctions_second = auctions[(auctions['date'] >= '2019-04-21 00:00:00.000000') & (auctions['date'] < '2019-04-24 00:00:00.000000')]
auctions_first_label = auctions_second.sort_values('date').groupby('ref_hash').first().reset_index()
auctions_first_label['time_appearence'] = (auctions_first_label['date'] - pd.to_datetime('2019-04-21 00:00:00.000000')).dt.seconds
auctions_first_label = auctions_first_label.drop(columns=['date', 'ref_type_id', 'source_id'])

In [6]:
auctions = 0

In [7]:
clicks = pd.read_csv("data/clicks.csv.gzip")

D:\Program Files\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (16,17) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [12]:
#installs = pd.read_csv("data/installs.csv.gzip")

D:\Program Files\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (4,15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [29]:
#events = pd.read_csv("data/events.csv.gzip")

D:\Program Files\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (13,20) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
target = pd.read_csv("data/target_final_competencia_revamped.csv")

In [8]:
clicks_first = clicks[clicks['created'] < '2019-04-21 00:00:00.000000']
clicks_first = pd.merge(clicks_first, clicks_first.groupby('ref_hash').agg('size').reset_index(), on='ref_hash', how='inner')
clicks_first = clicks_first.drop_duplicates('ref_hash')
clicks_first = clicks_first.rename(columns={0:'ammount_clicks'})

In [9]:
auctions = 0

In [9]:
clicks_first = clicks_first[['ref_hash', 'ammount_clicks']]

In [10]:
clicks_first.head()

,ref_hash,ammount_clicks
0,1293710398598742392,7
7,1663930990551616564,2
9,8488038938665586188,6
15,6488361690105189959,3
18,1348993302102753419,1


In [13]:
auctions_first = pd.merge(auctions_first, auctions_first.groupby('ref_hash').agg('size').reset_index(), on='ref_hash', how='inner')

In [33]:
auctions_first.head()

,date,ref_hash,ref_type_id,source_id,count,last_auction,first_auction,diff_auctions,prom_time_install
0,2019-04-20 23:57:27.912838,1109595589636746168,7,0,128,2019-04-20 23:59:50.195048,2019-04-18 00:00:38.281742,259151.913306,0
1,2019-04-20 23:59:34.748148,1109595589636746168,7,0,128,2019-04-20 23:59:50.195048,2019-04-18 00:00:38.281742,259151.913306,0
2,2019-04-18 20:26:06.818262,1109595589636746168,7,0,128,2019-04-20 23:59:50.195048,2019-04-18 00:00:38.281742,259151.913306,0
3,2019-04-18 20:25:17.123792,1109595589636746168,7,1,128,2019-04-20 23:59:50.195048,2019-04-18 00:00:38.281742,259151.913306,0
4,2019-04-20 23:59:50.195048,1109595589636746168,7,1,128,2019-04-20 23:59:50.195048,2019-04-18 00:00:38.281742,259151.913306,0


In [23]:
auctions_first['last_auction'] = auctions_first.groupby('ref_hash')['date'].transform('max')
auctions_first['first_auction'] = auctions_first.groupby('ref_hash')['date'].transform('min')

In [25]:
auctions_first['diff_auctions'] = (auctions_first['last_auction'] - auctions_first['first_auction']).dt.total_seconds()

In [32]:
auctions_first = auctions_first.rename(columns={0: 'count'})

In [36]:
auctions_first = auctions_first.drop(columns=['prom_time_install'])

In [35]:
auctions_first['prom_time_auction'] = 0
auctions_first.loc[auctions_first['count'] > 1, 'prom_time_auction'] = ((auctions_first['last_auction'] - auctions_first['first_auction'])/ \
                                 (auctions_first['count'] -1)).dt.total_seconds()

In [68]:
auctions_first['first_auction_sec'] = (auctions_first['first_auction'] - pd.Timestamp('2019-04-18 00:00:00.000000')).dt.total_seconds()
auctions_first['last_auction_sec'] = (auctions_first['last_auction'] - pd.Timestamp('2019-04-18 00:00:00.000000')).dt.total_seconds()

In [69]:
auctions_first.head()

,count,last_auction,first_auction,diff_auctions,prom_time_auction,first_auction_sec,last_auction_sec,ammount_clicks,time_appearence,ref_type_id_1,ref_type_id_7
0,128,2019-04-20 23:59:50.195048,2019-04-18 00:00:38.281742,259151.913306,2040.566247,38.281742,259190.195048,0.0,9,0,1
1,46,2019-04-20 23:57:36.225835,2019-04-20 01:46:34.133499,79862.092336,1774.713163,179194.133499,259056.225835,0.0,562,1,0
2,252,2019-04-20 23:59:29.271544,2019-04-18 06:27:50.996158,235898.275386,939.833766,23270.996158,259169.271544,0.0,26746,1,0
3,191,2019-04-20 23:57:30.262283,2019-04-18 04:02:46.305613,244483.956670,1286.757667,14566.305613,259050.262283,0.0,199,0,1
4,166,2019-04-20 23:57:28.868312,2019-04-18 00:35:31.040530,256917.827782,1557.077744,2131.040530,259048.868312,0.0,731,1,0


In [52]:
auctions_first = auctions_first.drop_duplicates('ref_hash')

In [53]:
auctions_first = pd.merge(auctions_first, clicks_first, on='ref_hash', how='left')
auctions_first = pd.merge(auctions_first, auctions_first_label, on='ref_hash', how='inner')
auctions_first['ref_type_id_1'] = auctions_first['ref_type_id'].apply(lambda x: 1 if x==1 else 0)
auctions_first['ref_type_id_7'] = auctions_first['ref_type_id'].apply(lambda x: 1 if x==7 else 0)

In [56]:
auctions_first = auctions_first.drop(columns=['date', 'ref_hash', 'source_id', 'ref_type_id', 'last_auction', 'first_auction'])

In [58]:
auctions_first.head()

,count,last_auction,first_auction,diff_auctions,prom_time_auction,first_auction_sec,last_auction_sec,ammount_clicks,time_appearence,ref_type_id_1,ref_type_id_7
0,128,2019-04-20 23:59:50.195048,2019-04-18 00:00:38.281742,259151.913306,2040.566247,0 days 00:00:38.281742,2 days 23:59:50.195048,NaN,9,0,1
1,46,2019-04-20 23:57:36.225835,2019-04-20 01:46:34.133499,79862.092336,1774.713163,2 days 01:46:34.133499,2 days 23:57:36.225835,NaN,562,1,0
2,252,2019-04-20 23:59:29.271544,2019-04-18 06:27:50.996158,235898.275386,939.833766,0 days 06:27:50.996158,2 days 23:59:29.271544,NaN,26746,1,0
3,191,2019-04-20 23:57:30.262283,2019-04-18 04:02:46.305613,244483.956670,1286.757667,0 days 04:02:46.305613,2 days 23:57:30.262283,NaN,199,0,1
4,166,2019-04-20 23:57:28.868312,2019-04-18 00:35:31.040530,256917.827782,1557.077744,0 days 00:35:31.040530,2 days 23:57:28.868312,NaN,731,1,0


In [59]:
auctions_first = auctions_first.fillna(0)

D:\Program Files\Anaconda\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Passing integers to fillna is deprecated, will raise a TypeError in a future version.  To retain the old behavior, pass pd.Timedelta(seconds=n) instead.
  """Entry point for launching an IPython kernel.


In [70]:
auctions_first.head()

,count,last_auction,first_auction,diff_auctions,prom_time_auction,first_auction_sec,last_auction_sec,ammount_clicks,time_appearence,ref_type_id_1,ref_type_id_7
0,128,2019-04-20 23:59:50.195048,2019-04-18 00:00:38.281742,259151.913306,2040.566247,38.281742,259190.195048,0.0,9,0,1
1,46,2019-04-20 23:57:36.225835,2019-04-20 01:46:34.133499,79862.092336,1774.713163,179194.133499,259056.225835,0.0,562,1,0
2,252,2019-04-20 23:59:29.271544,2019-04-18 06:27:50.996158,235898.275386,939.833766,23270.996158,259169.271544,0.0,26746,1,0
3,191,2019-04-20 23:57:30.262283,2019-04-18 04:02:46.305613,244483.956670,1286.757667,14566.305613,259050.262283,0.0,199,0,1
4,166,2019-04-20 23:57:28.868312,2019-04-18 00:35:31.040530,256917.827782,1557.077744,2131.040530,259048.868312,0.0,731,1,0


Time_appearence es el label, el valor a predecir por el modelo, los demás son features. Ref_type_id (one hot encoding) indica a que plataforma corresponde, appearences_in_auctions indica la cantidad de apariciones de dicho dispositivo en subastas y ammount_clicks indica la cantidad de clicks.

In [76]:
train, test = train_test_split(auctions_first, test_size=0.2)

In [77]:
train_Y = train['time_appearence']
train_X = train.drop(columns=['time_appearence'])
test_Y = test['time_appearence']
test_X = test.drop(columns=['time_appearence'])

In [149]:
model = tree.DecisionTreeRegressor(max_depth=50)

In [150]:
model = model.fit(train_X, train_Y)

In [151]:
mean_squared_error(train_Y, model.predict(train_X))

51919997.74649965

In [152]:
mean_squared_error(test_Y, model.predict(test_X))

1392476222.0248039

In [132]:
model.get_params()

{'criterion': 'mse',
 'max_depth': None,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'presort': False,
 'random_state': None,
 'splitter': 'best'}

In [198]:
xg_reg = xgb.XGBRegressor(objective ='reg:linear', colsample_bytree = 0.9, learning_rate = 0.1,
                max_depth = 40, alpha = 0, n_estimators = 20, gamma = 0.5)

In [199]:
xg_reg.fit(train_X, train_Y)

XGBRegressor(alpha=0, base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.9, gamma=0.5, learning_rate=0.1,
       max_delta_step=0, max_depth=40, min_child_weight=1, missing=None,
       n_estimators=20, n_jobs=1, nthread=None, objective='reg:linear',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=1)

In [200]:
mean_squared_error(train_Y, xg_reg.predict(train_X))

169689487.11835524

In [201]:
mean_squared_error(test_Y, xg_reg.predict(test_X))

833513312.203229

In [102]:
train_X.shape

(184538, 8)

In [103]:
test_X.shape

(46135, 8)